# Animations
This notebook produces the animations used in PendulumNB.

In [1]:
%matplotlib inline
from IPython.display import HTML
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.integrate import solve_ivp
from math import pi

In [2]:
# define some constants
g = 9.81
L = 9.81
omega = 2
k = 0.2
fps = 30

In [3]:
# define functions for pendulum equation and getting cartesian coordinates
def rhs(t, state, A):
    dxdt = np.zeros_like(state)
    dxdt[0] = state[1]
    dxdt[1] = -k*state[1] - (np.sin(state[0])/L)*(g + A*(omega**2)*np.cos(omega*t))
    return dxdt

def getCartesian(y, times, A):
    x = L*np.sin(y[0])
    y = -L*np.cos(y[0]) - A*np.cos(omega*times)
    return x, y

## Phase Space

In [4]:
def plotAngle(initial_state, t_init, t_fin, A, xlim=(-8, 8), ylim=(-11, 11), title="Simple Planar Pendulum"):
    t_eval = np.linspace(0, t_fin, t_fin*fps)
    args = A,
    sol = solve_ivp(rhs, (0, t_fin), initial_state, args=args, t_eval=t_eval, method='DOP853', rtol=1e-10, atol=1e-10)
    x, y = getCartesian(sol.y, t_eval, A)
    thetas = []
    times = []

    plt.close()
    fig = plt.figure(figsize=(12, 4))
    fig.suptitle(title)
    ax1 = fig.add_subplot(121, aspect='equal', autoscale_on=False,
                         xlim=xlim, ylim=ylim)
    ax2 = fig.add_subplot(122, autoscale_on=False,
                         xlim=(t_init, t_fin), ylim=(-5, 5))
    ax2.set_xlabel("Time")
    ax2.set_ylabel("Angle")
    ax1.grid()
    ax2.grid()

    pend, = ax1.plot([], [], 'o-', lw=2)
    graph, = ax2.plot([], [], lw=2)
    time_text = ax1.text(0.02, 0.95, '', transform=ax1.transAxes)

    def init():
        """initialize animation"""
        pend.set_data([], [])
        graph.set_data([], [])
        time_text.set_text('')
        return pend, time_text, graph

    def animate(i):
        """perform animation step"""
        thisx = [0, x[i]]
        thisy = [-A*np.cos(omega*t_eval[i]), y[i]]
        thetas.append(sol.y[0,i]*5)
        times.append(t_eval[i])
        pend.set_data(thisx, thisy)
        graph.set_data(times, thetas)
        time_text.set_text('time = %.1f' % t_eval[i])
        return pend, time_text, graph

    plt.close()
    ani = animation.FuncAnimation(fig, animate, frames=(t_fin-t_init)*fps, blit=True, init_func=init, interval=16)
    return ani

In [5]:
k = 0.0
ani = plotAngle([0.5, 0], 0, 20, 0.)
ani.save("angles.mp4")

In [6]:
def plotPhaseSpace(initial_state, t_init, t_fin, A, period, xlim=(-3.5, 3.5), ylim=(-3.5, 3.5), poincare=False):
    interval = period*fps
    t_eval = np.linspace(t_init, t_fin, int((t_fin-t_init)*fps))
    args = A,
    state = np.array([1., 0.])
    sol = solve_ivp(rhs, (0, t_fin), initial_state, args=args, t_eval=t_eval, method='DOP853', rtol=1e-10, atol=1e-10)
    x = ((sol.y[0] + pi) % (2*pi)) - pi
    y = sol.y[1]
    plotx = []
    ploty = []
    poincare_points = []
    
    plt.close()
    fig = plt.figure()
    fig.suptitle("Phase Space")
    ax = fig.add_subplot(111, aspect='equal', autoscale_on=False,
                         xlim=xlim, ylim=ylim)
    ax.set_xlabel("Theta")
    ax.set_ylabel("Theta dot")
    ax.grid()
    line, = ax.plot([], [], lw=2)
    lead_point = ax.scatter([], [], s=50, c='r')
    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
    if poincare:
        point = ax.scatter([], [], s=50, c='m')

    
    
    def init():
        """initialize animation"""
        line.set_data([], [])
        lead_point.set_offsets([])
        time_text.set_text('')
        if poincare:
            point.set_offsets([])
            return line, lead_point, point, time_text
        else:
            return line, lead_point, time_text
    
    def animate(i):
        plotx.append(x[i])
        ploty.append(y[i])
        line.set_data(plotx, ploty)
        lead_point.set_offsets(sol.y[:, i])
        time_text.set_text('time = %.1f' % t_eval[i])
        if poincare:
            if (i%interval == 0):
                point.set_offsets(sol.y[:, i])
                point.set_alpha(1)
            elif (i%interval < interval/9):
                point.set_alpha(1)
            elif (i%interval > interval/9):
                point.set_alpha(1 - (i%interval)/interval)
            return line, lead_point, point, time_text
        else:
            return line, lead_point, time_text
    
    plt.close()
    ani = animation.FuncAnimation(fig, animate, frames=int((t_fin-t_init)*fps), blit=True, init_func=init, interval=16)
    return ani

In [7]:
ani = plotPhaseSpace([0.5, 0.], 0, 4*pi, A=0., period=2*pi, xlim=(-1,1), ylim=(-1,1), poincare=True)
ani.save('PhaseSpaceSimple.mp4')

In [8]:
k = 0.2
ani = plotPhaseSpace([0.5, 0.], 0, 10*pi, A=0., period=2*pi, xlim=(-1,1), ylim=(-1,1))
ani.save('PhaseSpaceFriction.mp4')

## Moving the Pivot

In [9]:
def plot3Pendulum(initial_state, t_init, t_fin, A, xlim=(-8, 8), ylim=(-11, 11)):
    t_eval = np.linspace(0, t_fin, t_fin*fps)
    args1 = 0.1,
    args2 = 0.5,
    args3 = 1,
    
    sol1 = solve_ivp(rhs, (0, t_fin), initial_state, args=args1, t_eval=t_eval, method='DOP853', rtol=1e-10, atol=1e-10)
    x1, y1 = getCartesian(sol1.y, t_eval, 0.1)
    
    sol2 = solve_ivp(rhs, (0, t_fin), initial_state, args=args2, t_eval=t_eval, method='DOP853', rtol=1e-10, atol=1e-10)
    x2, y2 = getCartesian(sol2.y, t_eval, 0.5)
    
    sol3 = solve_ivp(rhs, (0, t_fin), initial_state, args=args3, t_eval=t_eval, method='DOP853', rtol=1e-10, atol=1e-10)
    x3, y3 = getCartesian(sol3.y, t_eval, 1)
    
    plt.close()
    fig = plt.figure(figsize=(8,5))
    
    ax1 = fig.add_subplot(131, aspect='equal', autoscale_on=False,
                         xlim=xlim, ylim=ylim)
    ax2 = fig.add_subplot(132, aspect='equal', autoscale_on=False,
                         xlim=xlim, ylim=ylim)
    ax3 = fig.add_subplot(133, aspect='equal', autoscale_on=False,
                         xlim=xlim, ylim=ylim)
    
    ax1.grid()
    ax1.set_title('A = 0.1')
    
    ax2.grid()
    ax2.set_title('A = 0.5')
    
    ax3.grid()
    ax3.set_title('A = 1')
    
    fig.tight_layout(pad=3.0)
    
    line1, = ax1.plot([], [], 'o-', lw=2)
    line2, = ax2.plot([], [], 'o-', lw=2)
    line3, = ax3.plot([], [], 'o-', lw=2)
    time_text = ax1.text(0.02, 0.95, '', transform=ax1.transAxes)
    
    def init():
        """initialize animation"""
        line1.set_data([], [])
        line2.set_data([], [])
        line3.set_data([], [])
        time_text.set_text('')
        return line1, line2, line3, time_text

    def animate(i):
        """perform animation step"""
        
        thisx1 = [0, x1[i]]
        thisy1 = [-0.1*np.cos(omega*t_eval[i]), y1[i]]
        line1.set_data(thisx1, thisy1)
        
        thisx2 = [0, x2[i]]
        thisy2 = [-0.5*np.cos(omega*t_eval[i]), y2[i]]
        line2.set_data(thisx2, thisy2)
        
        thisx3 = [0, x3[i]]
        thisy3 = [-1*np.cos(omega*t_eval[i]), y3[i]]
        line3.set_data(thisx3, thisy3)
        time_text.set_text('time = %.1f' % t_eval[i])
        return line1, line2, line3, time_text
    
    plt.close()
    ani = animation.FuncAnimation(fig, animate, frames=(t_fin-t_init)*fps, blit=True, init_func=init, interval=16)
    return ani

In [10]:
ani = plot3Pendulum([0.5, 0.], 0, 40, 1)
ani.save('steadymotion.mp4')

In [11]:
def PhaseSpaceRotating(initial_state, t_init, t_fin, A, period, xlim=(-3.5, 3.5), ylim=(-3.5, 3.5), poincare=False):
    interval = int(period*fps)
    t_eval = np.linspace(t_init, t_fin, int((t_fin-t_init)*fps))
    args = A,
    state = np.array([1., 0.])
    sol = solve_ivp(rhs, (0, t_fin), initial_state, args=args, t_eval=t_eval, method='DOP853', rtol=1e-10, atol=1e-10)
    x = ((sol.y[0] + pi) % (2*pi)) - pi
    y = sol.y[1]
    plotx = []
    ploty = []
    poincare_points = []
    
    plt.close()
    fig = plt.figure()
    fig.suptitle("Phase Space")
    ax = fig.add_subplot(111, aspect='equal', autoscale_on=False,
                         xlim=xlim, ylim=ylim)
    ax.set_xlabel("Theta")
    ax.set_ylabel("Theta dot")
    ax.grid()
    points = ax.scatter([], [], s=2)
    lead_point = ax.scatter([], [], s=50, c='r')
    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
    if poincare:
        point = ax.scatter([], [], s=50, c='m')

    
    
    def init():
        """initialize animation"""
        points.set_offsets([])
        lead_point.set_offsets([])
        time_text.set_text('')
        if poincare:
            point.set_offsets([])
            return points, lead_point, point, time_text
        else:
            return points, lead_point, time_text
    
    def animate(i):
        #print("i = " + str(i))
        plotx.append(x[i])
        ploty.append(y[i])
        points.set_offsets(np.column_stack((plotx, ploty)))
        lead_point.set_offsets(np.column_stack((x[i], y[i])))
        time_text.set_text('time = %.1f' % (t_eval[i] - t_init))
        if poincare:
            if (i%interval == 0):
                point.set_offsets(np.column_stack((x[i], y[i])))
                point.set_alpha(1)
            elif (i%interval < interval/9):
                point.set_alpha(1)
            elif (i%interval > interval/9):
                point.set_alpha(1 - (i%interval)/interval)
            return points, lead_point, point, time_text
        else:
            return points, lead_point, time_text
    
    plt.close()
    ani = animation.FuncAnimation(fig, animate, frames=int((t_fin-t_init)*fps), blit=True, init_func=init, interval=16)
    return ani

In [12]:
k = 0.2
ani = PhaseSpaceRotating([pi, 2], 200, 200 + 8*pi, 3.65, pi, xlim=(-3.5, 3.5), ylim=(-3.5, 3.5), poincare=True)
ani.save("period1.mp4")

In [13]:
ani = PhaseSpaceRotating([pi, 2], 200, 200 + 8*pi, 3.9, pi, xlim=(-3.5, 3.5), ylim=(-3.5, 3.5), poincare=True)
ani.save("period2.mp4")

In [14]:
def plotTwoPendula(initial_state1, initial_state2, t_init, t_fin, A, xlim=(-8, 8), ylim=(-11, 11)):
    t_eval = np.linspace(0, t_fin, t_fin*fps)
    args = A,
    
    sol1 = solve_ivp(rhs, (0, t_fin), initial_state1, args=args, t_eval=t_eval, method='DOP853', rtol=1e-10, atol=1e-10)
    x1, y1 = getCartesian(sol1.y, t_eval, A)
    
    sol2 = solve_ivp(rhs, (0, t_fin), initial_state2, args=args, t_eval=t_eval, method='DOP853', rtol=1e-10, atol=1e-10)
    x2, y2 = getCartesian(sol2.y, t_eval, A)
    
    pointsx1 = []
    pointsy1 = []
    pointsx2 = []
    pointsy2 = []
    
    plt.close()
    fig = plt.figure()
    ax = fig.add_subplot(111, aspect='equal', autoscale_on=False,
                         xlim=xlim, ylim=ylim)
    ax.grid()
    
    line1, = ax.plot([], [], 'o-', lw=2)
    points1 = ax.scatter([], [], s=1)

    line2, = ax.plot([], [], 'o-', lw=2)
    points2 = ax.scatter([], [], s=1)


    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
    #length_text = ax.text(0.02, 0.90, '', transform=ax.transAxes)
    
    def init():
        """initialize animation"""
        line1.set_data([], [])
        line2.set_data([], [])
        points1.set_offsets([])
        points2.set_offsets([])
        time_text.set_text('')
        return line1, line2, points1, time_text

    def animate(i):
        """perform animation step"""
        pointsx1.append(x1[i])
        pointsy1.append(y1[i])
        pointsx2.append(x2[i])
        pointsy2.append(y2[i])
        if i > 30:
            pointsx1.pop(0)
            pointsy1.pop(0)
            pointsx2.pop(0)
            pointsy2.pop(0)
            
        thisx1 = [0, x1[i]]
        thisy1 = [-A*np.cos(omega*t_eval[i]), y1[i]]
        thisx2 = [0, x2[i]]
        thisy2 = [-A*np.cos(omega*t_eval[i]), y2[i]]
        line1.set_data(thisx1, thisy1)
        line2.set_data(thisx2, thisy2)
        
        points1.set_offsets(np.column_stack((pointsx1, pointsy1)))
        points2.set_offsets(np.column_stack((pointsx2, pointsy2)))


        time_text.set_text('time = %.1f' % t_eval[i])
        return line1, line2, points1, time_text
    
    plt.close()
    ani = animation.FuncAnimation(fig, animate, frames=(t_fin-t_init)*fps, blit=True, init_func=init, interval=16)
    return ani

In [15]:
ani = plotTwoPendula([pi/2 + 0.0001745329, 0], [pi/2, 0], 0, 40, 5.15, xlim=(-17,17), ylim=(-17, 17))
ani.save("chaos.mp4")

## High Driving Frequencies

In [16]:
def plotPendulum(initial_state, t_init, t_fin, A, xlim=(-8, 8), ylim=(-11, 11)):
    t_eval = np.linspace(0, t_fin, t_fin*fps)
    args = A,
    sol = solve_ivp(rhs, (0, t_fin), initial_state, args=args, t_eval=t_eval, method='DOP853', rtol=1e-10, atol=1e-10)
    x, y = getCartesian(sol.y, t_eval, A)
    
    plt.close()
    fig = plt.figure()
    ax = fig.add_subplot(111, aspect='equal', autoscale_on=False,
                         xlim=xlim, ylim=ylim)
    ax.grid()
    
    line, = ax.plot([], [], 'o-', lw=2)
    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
    length_text = ax.text(0.02, 0.90, '', transform=ax.transAxes)
    
    def init():
        """initialize animation"""
        line.set_data([], [])
        time_text.set_text('')
        return line, time_text

    def animate(i):
        """perform animation step"""
        thisx = [0, x[i]]
        thisy = [-A*np.cos(omega*t_eval[i]), y[i]]
        line.set_data(thisx, thisy)
        time_text.set_text('time = %.1f' % t_eval[i])
        return line, time_text
    
    plt.close()
    ani = animation.FuncAnimation(fig, animate, frames=(t_fin-t_init)*fps, blit=True, init_func=init, interval=16)
    return ani



In [17]:
k=0.6
ani = plotPendulum([3.1, 0.], 0, 20, 0, xlim=(-10,10))
#HTML(ani.to_jshtml())
ani.save('FallingOver.mp4')

In [18]:
initial_theta = 130 
k=0.2
omega=90
ani = plotPendulum([initial_theta*pi/180., 0.], 0, 30, 0.2, xlim=(-10,10))
ani.save('stability.mp4')

In [19]:
def plotTwoPendula(initial_state1, initial_state2, t_init, t_fin, A, xlim=(-8, 8), ylim=(-11, 11)):
    t_eval = np.linspace(0, t_fin, t_fin*fps*16)
    args = A,
    
    sol1 = solve_ivp(rhs, (0, t_fin), initial_state1, args=args, t_eval=t_eval, method='DOP853', rtol=1e-10, atol=1e-10)
    x1, y1 = getCartesian(sol1.y, t_eval, A)
    
    sol2 = solve_ivp(rhs, (0, t_fin), initial_state2, args=args, t_eval=t_eval, method='DOP853', rtol=1e-10, atol=1e-10)
    x2, y2 = getCartesian(sol2.y, t_eval, A)
    
    pointsx1 = []
    pointsy1 = []
    pointsx2 = []
    pointsy2 = []
    
    plt.close()
    fig = plt.figure()
    ax = fig.add_subplot(111, aspect='equal', autoscale_on=False,
                         xlim=xlim, ylim=ylim)
    ax.grid()
    
    line1, = ax.plot([], [], 'o-', lw=2)
    points1 = ax.scatter([], [], s=1)

    line2, = ax.plot([], [], 'o-', lw=2)
    points2 = ax.scatter([], [], s=1)


    #length_text = ax.text(0.02, 0.90, '', transform=ax.transAxes)
    
    def init():
        """initialize animation"""
        line1.set_data([], [])
        line2.set_data([], [])
        points1.set_offsets([])
        points2.set_offsets([])
        return line1, line2, points1
    
    def animate(i):
        """perform animation step"""
        pointsx1.append(x1[i])
        pointsy1.append(y1[i])
        pointsx2.append(x2[i])
        pointsy2.append(y2[i])
        if i > 30:
            pointsx1.pop(0)
            pointsy1.pop(0)
            pointsx2.pop(0)
            pointsy2.pop(0)
            
        thisx1 = [0, x1[i]]
        thisy1 = [-A*np.cos(omega*t_eval[i]), y1[i]]
        thisx2 = [0, x2[i]]
        thisy2 = [-A*np.cos(omega*t_eval[i]), y2[i]]
        line1.set_data(thisx1, thisy1)
        line2.set_data(thisx2, thisy2)
        
        points1.set_offsets(np.column_stack((pointsx1, pointsy1)))
        points2.set_offsets(np.column_stack((pointsx2, pointsy2)))


        return line1, line2, points1
    
    plt.close()
    ani = animation.FuncAnimation(fig, animate, frames=(t_fin-t_init)*fps, blit=True, init_func=init, interval=33.3)
    return ani

In [20]:
ani = plotTwoPendula([pi/1.4 + 0.0001745329, 0], [pi/1.4, 0], 0, 20, 5, xlim=(-17,17), ylim=(-17, 17))
ani.save("chaos2.mp4")